In [ ]:
import os
import time
import numpy as np
# dashboard 
import ipywidgets as widgets
from IPython.display import display
# data pipeline
from blazingsql import BlazingContext
# dataviz
import holoviews as hv
import datashader as ds
from colorcet import fire
from holoviews import opts
import datashader.transfer_functions as tf
import holoviews.operation.datashader as hd
# from holoviews.element.tiles import EsriImagery
from holoviews.operation.datashader import datashade
hv.extension('bokeh')
# tag plot width & height
pw = 450
ph = 450
opts.defaults(opts.Scatter(width=pw, height=ph, color='blue'),
              opts.RGB(width=pw, height=ph),
              opts.Curve(width=pw))

# start up BlazingSQL
bc = BlazingContext()

# tag path to this notebook
cwd = os.getcwd()  
# add taxi wildcard to complete file path of data
path = cwd + '/data/nyc_taxi_jan15.csv'

# create base table
cols = ['Unnamed: 0', 'VendorID', 'tpep_pickup_datetime',
       'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'pickup_x',
       'pickup_y', 'RateCodeID', 'store_and_fwd_flag', 'dropoff_x',
       'dropoff_y', 'payment_type', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount']
col_types = ['int', 'int', 'date64', 'date64', 'int', 'float64', 'float64', 'float64', 
             'int', 'str', 'float64', 'float64', 'int', 'float64', 'float64', 'float64', 
             'float64', 'float64', 'float64', 'float64']
bc.create_table('big_taxi', path, names=cols, dtype=col_types)

# Firstly we need a common output for both dropdowns:
output = widgets.Output()
# Continuing the previous use-case, we will capture the plot in a new output variable:
plot_output = widgets.Output()
hv_output = widgets.Output()

# Here are the two dropdowns:
time_table = '''
             select
                 *,
                 month(tpep_pickup_datetime) months,
                 dayofmonth(tpep_pickup_datetime) dom,
                 year(tpep_pickup_datetime) years
             from
                 big_taxi
             '''

# value finder
ALL = 'ALL'
def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique

# month (currently only January)
dropdown_month = widgets.SelectMultiple(value=['1. Jan'],  # '2. Feb', '3. Mar', '4. Apr', '5. May', '6. Jun', '7. Jul', '8. Aug', '9. Sep', '10. Oct', '11. Nov', '12. Dec'], 
                                        options=['1. Jan'],  # '2. Feb', '3. Mar', '4. Apr', '5. May', '6. Jun', '7. Jul', '8. Aug', '9. Sep', '10. Oct', '11. Nov', '12. Dec'], 
                                        description='Months')  
# year (currently only 2015)
yr_vals = unique_sorted_values_plus_ALL(bc.sql(time_table).years)
yr_vals.remove('ALL')
yr_vals.remove(1969)
dropdown_year = widgets.SelectMultiple(value=yr_vals,
                                       options=yr_vals, 
                                       description='Years')  
# number of riders
dropdown_riders = widgets.Text(value='', 
                               placeholder='e.g. = 3 for 3 riders, blank for all',
                               description='# of Riders:')
# fare amount 
dropdown_fare = widgets.Text(value='',
                             placeholder='e.g. > 9 for over $9, blank for all',
                             description='Total Fare')
# location 
dropdown_location = widgets.ToggleButtons(value='Pickup Location',
                                          options=['Pickup Location', 'Dropoff Location'],
                                          description='Coordinates')
# A useful numeric widget is the BoundedFloatText; 
# we will give it a min, max and initial value, and the incremental step.
bounded_num = widgets.BoundedFloatText(min=0, max=100000, value=5, step=1)

# tag default dropdown values that affect SQL query adjustment
default_month = dropdown_month.value
default_year = dropdown_year.value
default_location = dropdown_location.value

# In order to colour the dataframe cells, we will define this function:
def colour_ge_value(value, comparison):
    if value >= comparison:
        return 'color: red'
    else:
        return 'color: black'

def common_filtering(month, year, n_riders, tol_cost, location, num):
    # clear outputs (replace instead of stacking, basically)
    output.clear_output()
    plot_output.clear_output()
    hv_output.clear_output()
    
    # affect sql: month, year, n_riders, tol_cost
    # affect visual dataframe: month, year, n_riders, tol_cost, num
    # affect plot: month, year, n_riders, tol_cost, location
    
    # check if input boxes are blank or not
    if n_riders != '':
        # and strip extra blank space from start & end
        n_riders = n_riders.strip()
        # check again that it's not blank
        if n_riders != '':
            # make sure there's a direction
            if n_riders[0] not in '<>=':
                # give user some time to figure it out
                time.sleep(5)                
                # we need a direction
                raise Exception(f'INVALID ARGUMENT ERROR:\nInput: {n_riders}\nIssue: number of riders input must start with one of the following:\n  >\n  <\n  =\n  >=\n  <=\nor be blank\n\nfor example: > 2 or = 3')
    if tol_cost != '':
        # e.g. '   ' = '' or '> 5 ' = '> 5'
        tol_cost = tol_cost.strip()
        # check again that it's not blank
        if tol_cost != '':
            # make sure there's a direction
            if tol_cost[0] not in '<>=':
                # give user some time to figure it out
                time.sleep(5)
                # we need a direction
                raise Exception(f'INVALID ARGUMENT ERROR:\nInput: {tol_cost}\nIssue: fare amount input must start with one of the following:\n  >\n  <\n  =\n  >=\n  <=\nor be blank\n\nfor example: > 2 or = 3')
    
    # start building our SQL query 
    query = 'select * from big_taxi'
    
    # has user focused specific months?
    if month != default_month:
        # go through months selected
        for m in list(month):
            # extract month number
            m = m.split('.')[0]
            # are months already being focused in the query?
            if 'months' in query:
                # add OR to include this month
                query += f' OR months = {m}'
            # ok, is query at all developed?
            elif 'WHERE' in query:
                # add AND statement for month 
                query = query + f' AND months = {m}'
            # WHERE is not in query yet
            else:
                # add WHERE & month to query 
                query += f' WHERE months = {m}'
    # has user focused specific years?
    if year != default_year:
        # go through years selected
        for y in list(year):
            # are years already being focused in the query?
            if 'years' in query:
                # add OR to include this year
                query += f' OR years = {y}'
            # ok, is query at all developed?
            elif 'WHERE' in query:
                # add AND statement for year 
                query = query + f' AND years = {y}'
            # WHERE is not in query yet
            else:
                # add WHERE & month to query 
                query += f' WHERE years = {y}'
    # has user focused specific rider counts? 
    if n_riders != '':
        # check how developed SQL statement is
        if 'WHERE' in query:
            # add AND statement for year 
            query = query + f' AND passenger_count {n_riders}'
        # WHERE is not in query yet
        else:
            # add WHERE & month to query 
            query += f' WHERE passenger_count {n_riders}'
    # has user focused specific fare values? 
    if tol_cost != '':
        # check how developed SQL statement is
        if 'WHERE' in query:
            # add AND statement for select total fares
            query = query + f' AND fare_amount {tol_cost}'
        # WHERE is not in query yet
        else:
            # add WHERE & total fare to query 
            query += f' WHERE fare_amount {tol_cost}'
    
    # run query & deliver cuDF DataFrame of results
    query = query.replace('big_taxi', f'({time_table})')
    gdf = bc.sql(query)
    
    with output:
        print(query)
        # apply the styling by calling the colour_ge_value function for the three numeric columns:
        display(common_filter.style.applymap(lambda x: colour_ge_value(x, num), 
                                             subset=['fare_amount', 'extra', 'mta_tax', 'tip_amount', 
                                                     'tolls_amount', 'improvement_surcharge', 'total_amount']))
    with plot_output:
        # and then we call the kdeplot method of seaborn by passing the number of visits:
        if dropdown_location.value == 'Pickup Location':
            agg = ds.Canvas(plot_width=pw, plot_height=ph).points(gdf, 'pickup_x', 'pickup_y')
        else:
            agg = ds.Canvas(plot_width=pw, plot_height=ph).points(gdf, 'dropoff_x', 'dropoff_y')
        display(tf.set_background(tf.shade(agg, cmap=fire),"black"))
        # print(query)
        
    with hv_output:
        scatter = hv.Scatter(gdf.to_pandas(), 'passenger_count', 'tip_amount')
        ranged = scatter.redim.range(tip_amount=(0,60), passenger_count=(-0.5,6.5))
        shaded = hd.spread(hd.datashade(ranged, x_sampling=0.15))
        labelled = shaded.redim.label(passenger_count="Passengers", tip_amount="Tip, $")
        display(labelled)
        
# The existing event handlers need to be adjusted 
# month, year, n_riders, tol_cost, location, num
def dropdown_month_eventhandler(change):
    # to pass the bounded_num.value:
    common_filtering(change.new, dropdown_year.value, dropdown_riders.value, 
                     dropdown_fare.value, dropdown_location.value, bounded_num.value)
def dropdown_year_eventhandler(change):
    # to pass the bounded_num.value:
    common_filtering(dropdown_month.value, change.new, dropdown_riders.value, 
                     dropdown_fare.value, dropdown_location.value, bounded_num.value)
def dropdown_riders_eventhandler(change):
    common_filtering(dropdown_month.value, dropdown_year.value, change.new, 
                     dropdown_fare.value, dropdown_location.value, bounded_num.value)    
def dropdown_fare_eventhandler(change):
    common_filtering(dropdown_month.value, dropdown_year.value, dropdown_riders.value, 
                     change.new, dropdown_location.value, bounded_num.value)    
def dropdown_location_eventhandler(change):
    common_filtering(dropdown_month.value, dropdown_year.value, dropdown_riders.value, 
                     dropdown_fare.value, change.new, bounded_num.value)    
# And finally we will plug-in the event handler of the new widget:
def bounded_num_eventhandler(change):
    common_filtering(dropdown_month.value, dropdown_year.value, dropdown_riders.value, 
                     dropdown_fare.value, dropdown_location.value, change.new)      

# We bind the handlers to the dropdowns, and that’s it!
dropdown_month.observe(dropdown_month_eventhandler, names='value')
dropdown_year.observe(dropdown_year_eventhandler, names='value')
dropdown_riders.observe(dropdown_riders_eventhandler, names='value')
dropdown_fare.observe(dropdown_fare_eventhandler, names='value')
dropdown_location.observe(dropdown_location_eventhandler, names='value')
bounded_num.observe(bounded_num_eventhandler, names='value')

# The HBox will add widgets to it one at a time from left-to-right:
input_widgets = widgets.HBox([dropdown_month, dropdown_year, dropdown_location])
input_widgets1 = widgets.HBox([dropdown_riders, dropdown_fare, bounded_num])

# dashboard display
tab = widgets.Tab([output, plot_output, hv_output])
tab.set_title(0, 'Dataset Exploration')
tab.set_title(1, 'Datashader Plot')
tab.set_title(2, 'HoloViews Plot')

In [ ]:
# dashboard inputs
display(input_widgets)
display(input_widgets1)

In [ ]:
# dashboard outputs 
display(tab)